# **Regularización En Regresión Lineal con Scikit-learn**


---
---

La regularización es una técnica para evitar el *overfitting* en los modelos, en este caso de regresión lineal, y con ello mejorar su generalización.

### **Regularización de Ridge (o regularización $L_2$)**

Dada la función costo escrita anteriormente en el notebook de [regresión lineal múltiple](https://github.com/Bronquivoide/Machine_Learning_Fundamentals/blob/main/Supervised%20Learning/Regresi%C3%B3n%20Lineal/Regresi%C3%B3n_Lineal_M%C3%BAltiple.ipynb), $\mathcal{L}(\vec{w}) = \frac{1}{N} \lVert \vec{f} - X \vec{w} \rVert ^2$, la regularización se logra añadiendo una penalización a los valores grandes de $\vec{w}$.

La función costo debe ser tal que los pesos asociados a $\vec{w}$ sean acotados mediante una constante:

$\vec{w} ^T \vec{w} \geq C$

Así, $\mathcal{L}(\vec{w})$ se puede reescribir de manera efectiva como:

$$
\mathcal{L}(\vec{w}) = \frac{1}{N} \lVert \vec{f} - X \vec{w} \rVert ^2 + α \lVert \vec{w} \rVert ^2
$$

En donde el término $α \lVert \vec{w} \rVert ^2$ es el que penaliza los valores grandes de $\vec{w}$.

A esta forma analítica de regularización se le conoce como **Regularización de Ridge** o **Regularización $L_2$**.

Con esta nueva $\mathcal{L}(\vec{w})$ se optimiza el vector de pesos, $\vec{w} ^* = ArgMin_{\vec{w}} \{ \mathcal{L}(\vec{w}) \}$, en donde tenemos que:

$$
\vec{w}^* = (X^T X - α \mathbb{I})^{-1}X^T \vec{f}
$$

El término $X^T X - α \mathbb{I}$ es una matriz regulada con la que $\vec{w}$ resuelve el overfitting y la generalización del modelo.


---

### **Otras regularizaciones**



-  **Regularización de Lasso o $L_1$:**

Aquí presenta la penalización en la función de costo mediante:

$$
\mathcal{L}(\vec{w}) = \frac{1}{N} \lVert \vec{f} - X \vec{w} \rVert ^2 + α \lVert \vec{w} \rVert
$$

-  **Regularización Elastic Net:**

Esta regularización combina $L_1$ y $L_2$ como sigue:

$$
\mathcal{L}(\vec{w}) = \frac{1}{N} \lVert \vec{f} - X \vec{w} \rVert ^2 + α _1 \lVert \vec{w} \rVert + α_2 \lVert \vec{w} \rVert ^2
$$


---

### **Regresión regularizada en `scikit-learn`**

$\circ$ Sintaxis de regresión Ridge:


```
from sklearn.linear_model import Ridge
ridge = Ridge(alpha)
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_test)
```

$\circ$ Sintaxis de regresión Lasso:


```
from sklearn.linear_model import Lasso
lasso = Lasso(alpha)
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_test)

```


$\circ$ Sintaxis de regresión Elastic Net:

```
from sklearn.linear_model import ElasticNet
elastic = ElasticNet(alpha, l1_ratio)
elastic.fit(X_train, y_train)
y_pred = elastic.predict(X_test)

```



Cuando se trata de un modelo de regresión regularizada, comunmente se [estandarizan](https://github.com/Bronquivoide/Machine_Learning_Fundamentals/blob/main/Preprocesamiento%20De%20Datos/Normalizaci%C3%B3n%20y%20Estandarizaci%C3%B3n/Normalizaci%C3%B3n_y_Estandarizaci%C3%B3n_de_Datos.ipynb) los datos que recibe.


---

### **Hiperparámetro $α$:**

El argumento `alpha` de los modelos anteriores es un valor numérico que corresponde al hiperparámetro $α$ que penaliza los pesos.

Lo que queremos es hallar el modelo Ridge, Lasso o elastic Net con el $α$ asociado que nos regrese la mejor $R^2$.  Para esto normalmente el hiperparámetro se ajusta de dos maneras:

**1) Se evaluan distintos valores de $α$**:

Suele tomarse una lista de `alpha` en un ciclo `for` para evaluar el rendimiento del modelo.

Por ejemplo, para una regularización Ridge:


```
from sklearn.linear_model import Ridge
scores = []
for alpha in [0.1, 1.0, 10.0, 100.0, 1000.0]:
   ridge = Ridge(alpha=alpha)    
   ridge.fit(X_train, y_train)
   y_pred = ridge.predict(X_test)
   scores.append(ridge.score(X_test, y_test))
print(scores)

```
Esto nos regresa una lista con los valores de $R^2$ asociados a cada `alpha`.

**2) Se ajusta $α$ mediante [$K-$Fold Cross-Validation ](https://github.com/Bronquivoide/Machine_Learning_Fundamentals/blob/main/Supervised%20Learning/Aprendizaje%20Supervisado%20con%20scikit-learn.pdf):**

La validación cruzada por $K$-folds busca el mejor valor de `alpha` (entre un conjunto predefinido o uno especificado mediante una lista) y entrena un modelo final con ese `alpha` óptimo, usando todos los datos de entrenamiento.

Por ejemplo, para una regularización Lasso:


```
from sklearn.linear_model import LassoCV
lasso = LassoCV(cv) #Un valor típico es cv=10
lasso.fit(X_train, y_train)
print("El mejor alpha es:", lasso.alpha_)
```

Así se regresa el modelo con el mejor rendimiento, es decir, con el mejor `alpha`. La sintaxis para Ridge es `import RidgeCV` y `import ElasticNetCV`.

---

### **Ejemplo: Regularización de Ridge para el dataset fetch_california_housing cardago en `scikit-learn`**

`fetch_california_housing()` es un conjunto de datos datos recolectados mediante un censo en California en 1990. Es muy popular para prácticas de regresión, cuya finalidad es predecir el valor de una vivienda a partir de características demográficas y geográficas.



In [35]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [26]:
#Cargando el dataset
cal_house=fetch_california_housing()
X, y=cal_house.data, cal_house.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Estandarización de los valores
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Ridge con k-fold C.V.
alphas=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
ridge_cv=RidgeCV(alphas=alphas, cv=10)
ridge_cv.fit(X_train_scaled, y_train)

print(f'El mejor modelo es el que tiene alpha={ridge_cv.alpha_}, pues tiene un R2-score de {round(ridge_cv.score(X_test_scaled, y_test),2)}')


El mejor modelo es el que tiene alpha=1.0, pues tiene un R2-score de 0.58


In [36]:
print(f'Los pesos asociados al modelo son: {ridge_cv.coef_}')
print('°°°°°°°°°°°°°°°°°°°')

print(f'Tenemos una matriz de diseño de orden {X.shape}, cuyas caracteríticas son: {cal_house.feature_names}')
print('°°°°°°°°°°°°°°°°°°°')


print("Valores reales:", y_test)
print('°°°°°°°°°°°°°°°°°°°')

print(f"Valores predichos por el modelo de Ridge:{ridge_cv.predict(X_test_scaled)}" )

Los pesos asociados al modelo son: [ 0.85432679  0.12262397 -0.29421036  0.33900794 -0.00228221 -0.04083302
 -0.89616759 -0.86907074]
°°°°°°°°°°°°°°°°°°°
Tenemos una matriz de diseño de orden (20640, 8), cuyas caracteríticas son: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
°°°°°°°°°°°°°°°°°°°
Valores reales: [0.477   0.458   5.00001 ... 5.00001 0.723   1.515  ]
°°°°°°°°°°°°°°°°°°°
Valores predichos por el modelo de Ridge:[0.71947224 1.76384666 2.709309   ... 4.46847645 1.18797174 2.00922052]
